## 🔹 Base Model Training — Algorithm Families Overview

Quyida loyihada qo‘llanilgan **asosiy algoritm oilalari** keltirilgan:

- 📏 **Boosting Model**
- 🌳 **Tree-Based Model**

In [38]:
import pandas as pd
import os
import numpy as np

In [39]:
os.chdir(r'C:\SML_Projects\SML_airplane_price_project')

In [40]:
x_train = pd.read_csv('data/split/x_train.csv')
x_test = pd.read_csv('data/split/x_test.csv')

y_train = pd.read_csv('data/split/y_train.csv')
y_test = pd.read_csv('data/split/y_test.csv')

In [41]:
y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [42]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9901 entries, 0 to 9900
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Model                        9901 non-null   object 
 1   Year_of_Manufacture          9901 non-null   int64  
 2   Number_of_Engines            9901 non-null   int64  
 3   Engine_Type                  9901 non-null   object 
 4   Capacity                     9901 non-null   int64  
 5   Fuel_Consumption_(L/hour)    9901 non-null   float64
 6   Hourly_Maintenance_Cost_($)  9901 non-null   float64
 7   Age                          9901 non-null   int64  
 8   Sales_Region                 9901 non-null   object 
 9   Price_($)                    9901 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 773.6+ KB


In [43]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2476 entries, 0 to 2475
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Model                        2476 non-null   object 
 1   Year_of_Manufacture          2476 non-null   int64  
 2   Number_of_Engines            2476 non-null   int64  
 3   Engine_Type                  2476 non-null   object 
 4   Capacity                     2476 non-null   int64  
 5   Fuel_Consumption_(L/hour)    2476 non-null   float64
 6   Hourly_Maintenance_Cost_($)  2476 non-null   float64
 7   Age                          2476 non-null   int64  
 8   Sales_Region                 2476 non-null   object 
 9   Price_($)                    2476 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 193.6+ KB


In [44]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

In [45]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import KNNImputer, SimpleImputer


# feature turlari
numeric_features = x_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = x_train.select_dtypes(include=['object']).columns

# =======================
# NUMERIC PIPELINE
# =======================
numeric_pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),   # numeric fill
    ('scaler', StandardScaler())
])

# =======================
# CATEGORICAL PIPELINE
# =======================
categorical_pipeline = Pipeline([
    ('encoder', OrdinalEncoder(
        handle_unknown='use_encoded_value',
        unknown_value=-1,
        encoded_missing_value=-2   # 👈 MANA SHU CAT NaN’NI HAL QILADI
    ))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])

# Extra Trees pipeline
et_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', ExtraTreesRegressor())
])

# Decision Tree pipeline
dt_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', DecisionTreeRegressor(random_state=42))
])

In [46]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9901 entries, 0 to 9900
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Model                        9901 non-null   object 
 1   Year_of_Manufacture          9901 non-null   int64  
 2   Number_of_Engines            9901 non-null   int64  
 3   Engine_Type                  9901 non-null   object 
 4   Capacity                     9901 non-null   int64  
 5   Fuel_Consumption_(L/hour)    9901 non-null   float64
 6   Hourly_Maintenance_Cost_($)  9901 non-null   float64
 7   Age                          9901 non-null   int64  
 8   Sales_Region                 9901 non-null   object 
 9   Price_($)                    9901 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 773.6+ KB


In [47]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2476 entries, 0 to 2475
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Model                        2476 non-null   object 
 1   Year_of_Manufacture          2476 non-null   int64  
 2   Number_of_Engines            2476 non-null   int64  
 3   Engine_Type                  2476 non-null   object 
 4   Capacity                     2476 non-null   int64  
 5   Fuel_Consumption_(L/hour)    2476 non-null   float64
 6   Hourly_Maintenance_Cost_($)  2476 non-null   float64
 7   Age                          2476 non-null   int64  
 8   Sales_Region                 2476 non-null   object 
 9   Price_($)                    2476 non-null   float64
dtypes: float64(3), int64(4), object(3)
memory usage: 193.6+ KB


In [48]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [49]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

### 📌 1. Boosting Models 
`ExtraTreesRegressor` 

In [50]:
et_pipeline.fit(x_train, y_train)
lr_y_pred = et_pipeline.predict(x_test)

et_score = r2_score(y_test, lr_y_pred)

et_mae = mean_absolute_error(y_test, lr_y_pred)
et_mse = mean_squared_error(y_test, lr_y_pred)
et_rmse = np.sqrt(np.mean((y_test - lr_y_pred) ** 2))

et_scores = cross_val_score(et_pipeline, x_train, y_train, cv=kf, scoring='r2')

print(f'Logistic Regression score: {et_score}')

print(f'\nMAE: {et_mae}')
print(f'MSE: {et_mse}')
print(f'RMSE: {et_rmse}')

print("\nK-Fold mean:", et_scores.mean())
print("K-Fold std:", et_scores.std()) 

Logistic Regression score: 0.9999997947476487

MAE: 0.056603392568659164
MSE: 6.229047859450728
RMSE: 2.4958060540536255

K-Fold mean: 0.9999998684416042
K-Fold std: 1.1949809685856655e-07


## 🌲 2. Tree-Based Models

`DecisionTreeRegressor`

In [51]:
dt_pipeline.fit(x_train, y_train)
dt_y_pred = dt_pipeline.predict(x_test)

dt_score = r2_score(y_test, dt_y_pred)

dt_mae = mean_absolute_error(y_test, dt_y_pred)
dt_mse = mean_squared_error(y_test, dt_y_pred)
dt_rmse = np.sqrt(np.mean((y_test - dt_y_pred) ** 2))

dt_scores = cross_val_score(dt_pipeline, x_train, y_train, cv=kf, scoring='r2')

print(f'DecisionTree score: {dt_score}')

print(f'\nMAE: {dt_mae}')
print(f'MSE: {dt_mse}')
print(f'RMSE: {dt_rmse}')

print("\nK-Fold mean:", dt_scores.mean())
print("K-Fold std:", dt_scores.std()) 

DecisionTree score: 1.0

MAE: 0.0
MSE: 0.0
RMSE: 0.0

K-Fold mean: 1.0
K-Fold std: 0.0


In [52]:
results = x_test[['Price_($)', 'Engine_Type', 'Capacity']].copy()
results['Range_(km)'] = y_test.values
results['lr_pred'] = lr_y_pred
results['dt_pred'] = dt_y_pred

results.sort_values("Range_(km)", ascending=False).head(20)

,Price_($),Engine_Type,Capacity,Range_(km),lr_pred,dt_pred
40,4.019874e+08,Turbofan,396,15600,15600.0,15600.0
2469,3.944497e+08,Turbofan,396,15600,15600.0,15600.0
16,7.302247e+08,Turbofan,396,15600,15600.0,15600.0
33,3.293284e+08,Turbofan,396,15600,15600.0,15600.0
2437,3.544446e+08,Turbofan,396,15600,15600.0,15600.0
2438,5.534876e+08,Turbofan,396,15600,15600.0,15600.0
2439,3.849041e+08,Turbofan,396,15600,15600.0,15600.0
2444,6.969825e+08,Turbofan,396,15600,15600.0,15600.0
2467,4.008150e+08,Turbofan,396,15600,15600.0,15600.0
817,5.537841e+08,Turbofan,396,15600,15600.0,15600.0


In [53]:
from rich.table import Table
from rich.console import Console

console = Console()

results = [
    # Boosting Family
    ['ExtraTrees', et_score, et_mae, et_mse, et_rmse, et_scores.mean(), et_scores.std()],

    # Tree-Based Family
    ['DecisionTree', dt_score, dt_mae, dt_mse, dt_rmse, dt_scores.mean(), dt_scores.std()],
]

result_sorted = sorted(results, key=lambda i: i[1], reverse=True)
best_model = max(results, key=lambda x: x[1])

table = Table(title="Base model training Comparison", show_lines=True)
table.add_column("Algorithm")
table.add_column("R2_Score")
table.add_column("MAE")
table.add_column("MSE")
table.add_column("RMSE")
table.add_column("K-Fold mean")
table.add_column("K-Fold std")

for row in result_sorted:
    algo, r2, mae, mse, rmse, kmean, kstd = row

    if row == best_model:
        table.add_row(
            f"[bold green]{algo}[/bold green]",
            f"[bold green]{r2:.2f}[/bold green]",
            f"[bold green]{mae:.2f}[/bold green]",
            f"[bold green]{mse:.2f}[/bold green]",
            f"[bold green]{rmse:.2f}[/bold green]",
            f"[bold green]{kmean:.2f}[/bold green]",
            f"[bold green]{kstd:.2f}[/bold green]",
        )
    else:
        table.add_row(algo, f"{r2:.2f}", f"{mae:.2f}", f"{mse:.2f}", f"{rmse:.2f}", f"{kmean:.2f}", f"{kstd:.2f}")

console.print(table)

                      Base model training Comparison                       
┏━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Algorithm    ┃ R2_Score ┃ MAE  ┃ MSE  ┃ RMSE ┃ K-Fold mean ┃ K-Fold std ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ DecisionTree │ 1.00     │ 0.00 │ 0.00 │ 0.00 │ 1.00        │ 0.00       │
├──────────────┼──────────┼──────┼──────┼──────┼─────────────┼────────────┤
│ ExtraTrees   │ 1.00     │ 0.06 │ 6.23 │ 2.50 │ 1.00        │ 0.00       │
└──────────────┴──────────┴──────┴──────┴──────┴─────────────┴────────────┘

In [54]:
os.makedirs('results', exist_ok=True)

temp_console = Console(record=True)
temp_console.print(table)
text = temp_console.export_text()
with open('results/baseline_model.txt', 'w', encoding='utf-8') as f:
    f.write(text)

                      Base model training Comparison                       
┏━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Algorithm    ┃ R2_Score ┃ MAE  ┃ MSE  ┃ RMSE ┃ K-Fold mean ┃ K-Fold std ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ DecisionTree │ 1.00     │ 0.00 │ 0.00 │ 0.00 │ 1.00        │ 0.00       │
├──────────────┼──────────┼──────┼──────┼──────┼─────────────┼────────────┤
│ ExtraTrees   │ 1.00     │ 0.06 │ 6.23 │ 2.50 │ 1.00        │ 0.00       │
└──────────────┴──────────┴──────┴──────┴──────┴─────────────┴────────────┘

In [55]:
import joblib

os.makedirs("pipeline", exist_ok=True)

model_path = f"pipeline/full_pipeline.joblib"
joblib.dump(dt_pipeline, model_path)

print(f"✅ Best model saved as: {model_path}")

✅ Best model saved as: pipeline/full_pipeline.joblib


# Next step - Feature selection

<a href="FS/without_fs.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        Without Feature selection 
    </button>
</a>

<a href="FS/filter.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        Filter method
    </button>
</a>

<a href="FS/embedded.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        Embedded method
    </button>
</a>